In [ ]:
import numpy as np
import wave
import warnings
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA
from scipy.io import wavfile

%matplotlib inline

warnings.filterwarnings('ignore')

In [ ]:
# open and read wav files
mix_wave_1 = wave.open('contagem_sep1.wav', 'r')
mix_wave_2 = wave.open('contagem_sep2.wav', 'r')
mix_wave_3 = wave.open('contagem_sep3.wav', 'r')

In [ ]:
# get parameters of wav file
mix_wave_1.getparams()

In [ ]:
# get length of wav file in seconds
len_of_file = mix_wave_1.getparams().nframes/48000 # nframes / framerate
print(f"length of file in seconds : {len_of_file}")

In [ ]:
# extract number of frames
def extract_frames(file):
  signal_raw = file.readframes(-1)
  signal = np.fromstring(signal_raw, dtype=np.short)
  return signal

In [ ]:
signal_1 = extract_frames(mix_wave_1)
signal_2 = extract_frames(mix_wave_2)
signal_3 = extract_frames(mix_wave_3)

In [ ]:
len_of_signal_1 = len(signal_1)
print(f"length : {len_of_signal_1}")

In [ ]:
# get timing of signal_1
def get_timing(file, signal):
  fs = file.getframerate()
  timing = np.linspace(0, len(signal)/fs, num=len(signal))
  return timing

In [ ]:
timing_1 = get_timing(mix_wave_1, signal_1)
timing_2 = get_timing(mix_wave_2, signal_2)
timing_3 = get_timing(mix_wave_3, signal_3)

In [ ]:
def plot_wave(timing, signal, title):
  plt.figure(figsize=(12 ,2))
  plt.title(title)
  plt.plot(timing, signal)
  plt.show()

In [ ]:
plot_wave(timing_1, signal_1, 'Recording 1')
plot_wave(timing_2, signal_2, "Recording 2")
plot_wave(timing_3, signal_3, "Recording 3")

In [ ]:
# zip all the signal into single list
data =  list(zip(signal_1, signal_2, signal_3))

In [ ]:
# see what data looks like
data[:10]

In [ ]:
# initialise FastICA and fit and transform data
fastica = FastICA(n_components=3)
#fastica = FastICA(n_components=2)
ica_result = fastica.fit_transform(data)

In [ ]:
# print the shape of result
ica_result.shape

In [ ]:
# split signals
result_signal_1 = ica_result[:, 0]
result_signal_2 = ica_result[:, 1]
result_signal_3 = ica_result[:, 2]

In [ ]:
# function to plot individual components
def plot_result_signal(result_signal, title):
  plt.figure(figsize=(12, 2))
  plt.title(title)
  plt.plot(result_signal)
  plt.ylim(-0.010, 0.010)
  plt.show()

In [ ]:
plot_result_signal(result_signal_1, "Independent component 1")
plot_result_signal(result_signal_2, "Independent component 2")
plot_result_signal(result_signal_3, "Independent component 2")

In [ ]:
# convert signal to int16
def convert_to_int16(result_signal, fs, filename):
  result_signal_int = np.short(result_signal * 32767 * 100)
  wavfile.write(filename, fs, result_signal_int)

In [ ]:
# get framerate
fs_1 = mix_wave_1.getframerate()
fs_2 = mix_wave_2.getframerate()
fs_3 = mix_wave_3.getframerate()

In [ ]:
# convert back to wav
convert_to_int16(result_signal_1, fs_1, "contagem_separado1.wav")
convert_to_int16(result_signal_2, fs_2, "contagem_separado2.wav")
convert_to_int16(result_signal_3, fs_2, "contagem_separado3.wav")